In [2]:
import pandas as pd
import requests

## URL with Table (easier extraction)

In [11]:
url = 'https://www.worldcoinindex.com/'
crypto_url = requests.get(url)
crypto_url

<Response [200]>

In [13]:
body = crypto_url.text

In [14]:
crypto_data = pd.read_html(body)
print(type(crypto_data))
print(len(crypto_data))

<class 'list'>
1


In [15]:
crypto_data = crypto_data[0]
crypto_data.head()

,#,Unnamed: 1,Name,Ticker,Last price,%,24 high,24 low,Price Charts 7d,24 volume,# Coins,Market cap
0,1,NaN,Ethereum,ETH,"$ 2,796.37",+3.28%,"$ 2,890.34","$ 2,664.86",NaN,$ 16.25B,116.14M,$ 324.77B
1,2,NaN,Bitcoin,BTC,"$ 38,576",+2.66%,"$ 39,475","$ 37,215",NaN,$ 11.59B,18.72M,$ 722.38B
2,3,NaN,Dogecoin,DOGE,$ 0.389339,-7.97%,$ 0.440510,$ 0.378179,NaN,$ 6.98B,129.40B,$ 50.38B
3,4,NaN,Binancecoin,BNB,$ 414.54,+3.25%,$ 432.45,$ 391.85,NaN,$ 3.04B,154.53M,$ 64.05B
4,5,NaN,Ripple,XRP,$ 1.03,+0.64%,$ 1.08,$ 1.01,NaN,$ 2.80B,46.15B,$ 47.59B


## URL without Table (harder extraction)

In [38]:
page = requests.get('https://dataquestio.github.io/web-scraping-pages/simple.html')
page.status_code
print(page.text)

<!DOCTYPE html>
<html>
    <head>
        <title>A simple example page</title>
    </head>
    <body>
        <p>Here is some simple content for this page.</p>
    </body>
</html>


In [21]:
from bs4 import BeautifulSoup

In [59]:
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [63]:
list(soup.children)

['html',
 '\n',
 <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [41]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [68]:
html = list(soup.children)[2]

In [49]:
list(html.children)

['\n',
 <head>
 <title>A simple example page</title>
 </head>,
 '\n',
 <body>
 <p>Here is some simple content for this page.</p>
 </body>,
 '\n']

In [50]:
body = list(html.children)[3]

In [51]:
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [52]:
p = list(body.children)[1]

In [58]:
p.text

'Here is some simple content for this page.'

In [72]:
soup = BeautifulSoup(page.text, 'html.parser')
soup.find_all('p')[0].text

'Here is some simple content for this page.'

In [73]:
soup.find('p')

<p>Here is some simple content for this page.</p>

## Searching for tags by class and id

In [76]:
page = requests.get('https://dataquestio.github.io/web-scraping-pages/ids_and_classes.html')
soup = BeautifulSoup(page.text, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [78]:
soup.find_all('p', class_ = 'outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [79]:
soup.find_all(class_ = 'outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [80]:
soup.find_all(id = 'first')

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

## Using CSS Selectors

* p a — finds all a tags inside of a p tag.
* body p a — finds all a tags inside of a p tag inside of a body tag.
* html body — finds all body tags inside of an html tag.
* p.outer-text — finds all p tags with a class of outer-text.
* p#first — finds all p tags with an id of first.
* body p.outer-text — finds any p tags with a class of outer-text inside of a body tag.

In [81]:
soup.select('div p')

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>,
 <p class="inner-text">
                 Second paragraph.
             </p>]

# Weather Data

In [144]:
weather_url = requests.get('https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168')
soup = BeautifulSoup(weather_url.text, 'html.parser')
seven_day = soup.find(id = 'seven-day-forecast')
forecast_items = seven_day.find_all(class_ = 'tombstone-container')
tonight = forecast_items[1]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Increasing clouds, with a low around 54. West wind 16 to 21 mph, with gusts as high as 28 mph. " class="forecast-icon" src="newimages/medium/nsct.png" title="Tonight: Increasing clouds, with a low around 54. West wind 16 to 21 mph, with gusts as high as 28 mph. "/>
 </p>
 <p class="short-desc">
  Increasing
  <br/>
  Clouds
 </p>
 <p class="temp temp-low">
  Low: 54 °F
 </p>
</div>


In [176]:
period = tonight.find('p', class_ = 'period-name').text
short_desc = tonight.find(class_ = 'short-desc').text
temp = tonight.find(class_ = 'temp').text
print(period)
print(short_desc)
print(temp)

Tonight
IncreasingClouds
Low: 54 °F
